<a href="https://colab.research.google.com/github/Anu-kriti/AIAgent/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.9.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="service_law_training_master.jsonl")["train"]

# Inspect a sample
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'system', 'content': 'You are a legal AI assistant trained on Indian service law.'}, {'role': 'user', 'content': 'What is the significance of the Factories Act, 1881 in Indian service law?'}, {'role': 'assistant', 'content': 'The Factories Act of 1881 was one of the earliest labor regulations introduced during the colonial period. It focused on working conditions, especially working hours, child labor, and safety measures in factories. Although limited and employer-centric, it marked the beginning of statutory labor protections and laid the foundation for later reforms in Indian service law.'}]}


In [5]:
tokenizer.chat_template = """{% for message in messages %}
{{ message['role'].capitalize() }}: {{ message['content'] }}
{% endfor %}"""

In [6]:
def formatting_func(example):
    return tokenizer.apply_chat_template(example["messages"], tokenize=False)

dataset = dataset.map(lambda x: {"text": formatting_func(x)})

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

In [ ]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }
# pass

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/39 [00:00<?, ? examples/s]

In [8]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.881 GB of memory reserved.


In [10]:
# Total parameters
print("Total parameters:", sum(p.numel() for p in model.parameters()))

# Trainable parameters (should be small if LoRA still attached)
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))




Total parameters: 4670623744
Trainable parameters: 41943040


In [14]:
(41943040/4670623744)

0.008980179586052307

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 39 | Num Epochs = 12 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.653900
2,2.527300
3,2.623300
4,2.370300
5,2.101000
6,2.021600
7,1.529100
8,1.493500
9,1.439800
10,1.469400


In [16]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

324.1245 seconds used for training.
5.4 minutes used for training.
Peak reserved memory = 6.881 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 46.679 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [18]:
# Total parameters
print("Total parameters:", sum(p.numel() for p in model.parameters()))

# Trainable parameters (should be small if LoRA still attached)
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))




Total parameters: 4670623744
Trainable parameters: 41943040


In [19]:
import math
eval_results = trainer.evaluate()
print("Eval loss:", eval_results["eval_loss"])
print("Perplexity:", math.exp(eval_results["eval_loss"]))


ValueError: Trainer: evaluation requires an eval_dataset.

In [22]:
from bert_score import score
P, R, F1 = score(
    ["Prayer: It is respectfully prayed that this Hon’ble Tribunal may..."],  # candidate
    ["Prayer: It is most respectfully prayed that this Hon’ble Tribunal may..."],  # reference
    lang="en"
)
print("BERTScore F1:", F1.mean().item())


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.992339551448822


In [ ]:
Establish guardrails and safety mechanisms to prevent harmful or biased
outputs.

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [23]:
# Merge LoRA into the base (currently loaded in RAM)
merged_model = model.merge_and_unload()

# Save full model
save_dir = "service_law_model_full"
merged_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('service_law_model_full/tokenizer_config.json',
 'service_law_model_full/special_tokens_map.json',
 'service_law_model_full/chat_template.jinja',
 'service_law_model_full/tokenizer.json')

In [43]:
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import BitsAndBytesConfig


# Base model you fine-tuned on
base_model = "unsloth/Meta-Llama-3.1-8B"
lora_path = "outputs/checkpoint-60"   # directory with your LoRA fine-tuned weights


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",   # safe on T4
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    base_model,
    load_in_4bit=True,
    # quantization_config=bnb_config, # Removed explicit quantization_config
    device_map={"": "cuda"},
)

# Attach LoRA weights
model = PeftModel.from_pretrained(model, lora_path)
# Merge LoRA into the base model
merged_model = model.merge_and_unload()

# Save as a standalone model
save_dir = "merged_service_law_model"
merged_model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [46]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [48]:
model, tokenizer = FastLanguageModel.from_pretrained(
    "service_law_model_full",
    max_seq_length=2048,
    load_in_4bit=True,   # re-quantize on load
    device_map="cuda", # Explicitly load to cuda
)

==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [63]:
# import gc
# import torch

# # Delete large objects
# if 'model' in locals():
#     del model
# if 'tokenizer' in locals():
#     del tokenizer
# if 'trainer' in locals(): # if trainer is still in memory
#     del trainer


# # Run garbage collection
# gc.collect()
# torch.cuda.empty_cache()

In [34]:
!nvidia-smi

Thu Sep  4 22:12:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P0             28W /   70W |    7568MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [26]:
# chat = [
#     {"role": "system", "content": "You are a legal AI assistant specialized in Indian service law."},
#     {"role": "user", "content": "Draft a short prayer for reinstatement after wrongful termination."},
# ]

# # 1. Get the full prompt as text
# prompt = tokenizer.apply_chat_template(chat, tokenize=False)

# # 2. Tokenize → dict with input_ids & attention_mask
# # Remove .to("cpu") so inputs are on the correct device (cuda)
# inputs = tokenizer(prompt, return_tensors="pt")

# # 3. Generate
# outputs = model.generate(
#     **inputs,
#     max_new_tokens=300,
#     temperature=0.7,
#     top_p=0.9,
#     do_sample=True,
# )

# # 4. Decode
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

RuntimeError: Expected all tensors to be on the same device, but got index is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA__index_select)

In [37]:
merged_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [33]:
tokenizer = AutoTokenizer.from_pretrained("service_law_model_full")
model = AutoModelForCausalLM.from_pretrained("service_law_model_full", device_map="auto", torch_dtype="auto")


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [32]:
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Path to your merged model folder
model_path = "service_law_model_full"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",       # Automatically map to GPU/CPU
    torch_dtype=torch.float16, # FP16 is safe for most GPUs (use "auto" if unsure)
)

# Example inference
prompt = "Draft a short prayer for reinstatement after wrongful termination."

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=300,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import gradio as gr

def chat_fn(message, history):
    chat = [{"role": "system", "content": "You are a legal AI assistant for Indian service law."}]
    for h in history:
        chat.append({"role": "user", "content": h[0]})
        chat.append({"role": "assistant", "content": h[1]})
    chat.append({"role": "user", "content": message})

    inputs = tokenizer.apply_chat_template(chat, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=400, temperature=0.7, top_p=0.9)
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return reply

gr.ChatInterface(fn=chat_fn).launch()


In [ ]:
# # alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence.", # instruction
#         "1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# # alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence.", # instruction
#         "1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>


# RAG

In [2]:
!pip install sentence-transformers faiss-cpu transformers unsloth peft datasets PyPDF2 bert-score
# If you have CUDA and want faiss-gpu:
# pip install faiss-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Fou

In [3]:
"""
RAG pipeline for Service-Law Llama model (Llama-3.1).
- Builds FAISS index from text chunks (PDFs/plain text)
- Retrieves top-k docs for a query
- Constructs a prompt combining system + retrieved contexts + user query
- Generates answer using merged Llama model (FastLanguageModel or AutoModel)
- Evaluates retrieval precision and lexical overlap; optionally BERTScore
"""

import os
import glob
import math
import faiss
import torch
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
from peft import PeftModel  # only if you plan to attach adapters
import PyPDF2

# ---------- CONFIG ----------
# point these to your actual files
MERGED_MODEL_DIR = "service_law_model_full"   # folder with merged model saved_pretrained()
TOKENIZER_DIR = MERGED_MODEL_DIR

DOCUMENT_PDF = "/content/The-Evolution-of-Service-Law-in-India.pdf"  # example; you can process many files
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
FAISS_INDEX_PATH = "service_law_faiss.index"
DOC_STORE_PATH = "service_law_docs.jsonl"  # will store chunk metadata
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMB_DIM = 384  # all-MiniLM-L6-v2 produces 384-d vectors
CHUNK_SIZE = 800         # characters per chunk
CHUNK_OVERLAP = 120      # overlap for chunking
TOP_K = 5

# ---------- UTIL: load & chunk PDF ----------
def extract_texts_from_pdf(pdf_path: str) -> List[str]:
    texts = []
    reader = PyPDF2.PdfReader(pdf_path)
    for p in reader.pages:
        try:
            texts.append(p.extract_text() or "")
        except Exception:
            texts.append("")
    full = "\n".join(texts)
    return [full]  # return full text; we'll chunk below

def chunk_text(text: str, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP) -> List[str]:
    chunks = []
    start = 0
    L = len(text)
    while start < L:
        end = min(start + chunk_size, L)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end - overlap
        if start < 0: start = 0
        if start >= L:
            break
    return chunks

# ---------- Build embeddings & FAISS index ----------
def build_index_from_pdf(pdf_path: str,
                         embedding_model_name: str = EMBEDDING_MODEL_NAME,
                         index_path: str = FAISS_INDEX_PATH,
                         doc_store_path: str = DOC_STORE_PATH):
    # 1. extract text
    pages = extract_texts_from_pdf(pdf_path)
    text = "\n".join(pages)
    chunks = chunk_text(text)

    # 2. embed
    embedder = SentenceTransformer(embedding_model_name, device=DEVICE)
    embeddings = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True)

    # 3. build faiss index
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # inner product (use cosine by normalizing)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    # 4. save index and docs
    faiss.write_index(index, index_path)

    # store metadata: a simple jsonl: id, text
    import json
    with open(doc_store_path, "w", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            f.write(json.dumps({"id": i, "text": chunk}, ensure_ascii=False) + "\n")

    print(f"Built FAISS index: {index.ntotal} chunks")
    return index, chunks

# ---------- Load index & docs ----------
def load_index_and_docs(index_path: str = FAISS_INDEX_PATH, doc_store_path: str = DOC_STORE_PATH):
    index = faiss.read_index(index_path)
    docs = []
    import json
    with open(doc_store_path, "r", encoding="utf-8") as f:
        for line in f:
            docs.append(json.loads(line.strip()))
    return index, docs

# ---------- Retriever ----------
def retrieve(query: str, embedder: SentenceTransformer, index: faiss.Index, docs: List[Dict], top_k: int = TOP_K):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    scores, idxs = index.search(q_emb, top_k)
    results = []
    for i, idx in enumerate(idxs[0]):
        if idx < 0 or idx >= len(docs): continue
        results.append({"id": docs[idx]["id"], "text": docs[idx]["text"], "score": float(scores[0][i])})
    return results

# ---------- Prompt construction ----------
def build_rag_prompt(retrieved: List[Dict], user_query: str) -> str:
    # System + retrieved contexts + user question
    system = "You are a legal AI assistant specialized in Indian service law. Use the retrieved passages as the only sources for facts — do not hallucinate."
    context = "\n\n".join([f"[DOC {r['id']} | score:{r['score']:.3f}]\n{r['text']}" for r in retrieved])
    prompt = f"{system}\n\nRetrieved Contexts:\n{context}\n\nUser Query:\n{user_query}\n\nAnswer (ground your answer in the retrieved contexts; cite DOC ids where applicable):"
    return prompt

# ---------- Load generator model (Unsloth FastLanguageModel) ----------
def load_generator(model_dir: str = MERGED_MODEL_DIR, tokenizer_dir: str = TOKENIZER_DIR):
    # Try Unsloth FastLanguageModel, fallback to AutoModelForCausalLM if needed
    try:
        model, tokenizer = FastLanguageModel.from_pretrained(model_dir, max_seq_length=1024, load_in_4bit=True)
        FastLanguageModel.for_inference(model)
        print("Loaded Unsloth FastLanguageModel (quantized).")
        return model, tokenizer
    except Exception as e:
        print("Unsloth load failed, trying transformers AutoModel (may be heavier):", e)
        from transformers import AutoModelForCausalLM, AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir, use_fast=False)
        model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", torch_dtype=torch.float16)
        print("Loaded AutoModelForCausalLM.")
        return model, tokenizer

# ---------- Generation (chat-style prompt) ----------
def generate_answer_from_prompt(model, tokenizer, prompt: str, max_new_tokens=256):
    # If tokenizer has chat_template, use it; else feed raw prompt
    try:
        # apply_chat_template requires a messages structure; we have a raw prompt string so do raw
        inputs = tokenizer(prompt, return_tensors="pt")
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    except Exception:
        # fallback - tokenize raw
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(DEVICE)

    # For PEFT PeftModel, ensure generate works
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.0,    # deterministic for evaluation; adjust for creativity
        top_p=0.95,
        do_sample=False,
        use_cache=True,
    )
    # decode
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

# ---------- Simple evaluation metrics ----------
def lexical_overlap_score(generated: str, retrieved_texts: List[str]) -> float:
    # fraction of generated tokens that also appear in retrieved contexts (naive)
    gen_tokens = set(generated.lower().split())
    context_tokens = set(" ".join(retrieved_texts).lower().split())
    if not gen_tokens:
        return 0.0
    overlap = gen_tokens & context_tokens
    return len(overlap) / len(gen_tokens)

# Optional: BERTScore if installed (more robust)
def bertscore_f1(cands: List[str], refs: List[str]):
    from bert_score import score
    P, R, F1 = score(cands, refs, lang="en", verbose=False)
    return F1.mean().item()

# ---------- End-to-end test harness ----------
def run_rag_test(queries: List[Dict], index, docs, embedder, model, tokenizer, top_k=TOP_K):
    results = []
    for q in queries:
        query_text = q["query"]
        gold = q.get("gold", None)  # optional reference answer
        retrieved = retrieve(query_text, embedder, index, docs, top_k=top_k)
        prompt = build_rag_prompt(retrieved, query_text)
        gen = generate_answer_from_prompt(model, tokenizer, prompt)
        score_lex = lexical_overlap_score(gen, [r["text"] for r in retrieved])
        results.append({"query": query_text, "generated": gen, "retrieved": retrieved, "lexical_overlap": score_lex, "gold": gold})
        print("=== QUERY ===\n", query_text)
        print("=== RETRIEVED (ids) ===", [r["id"] for r in retrieved])
        print("=== GENERATED ===\n", gen[:1000])
        print("LEXICAL_OVERLAP:", score_lex)
        print("-" * 80)
    return results

# ----------------- MAIN FLOW (example run) -----------------
if __name__ == "__main__":
    # 1) Build index from the PDF (run once)
    if not os.path.exists(FAISS_INDEX_PATH) or not os.path.exists(DOC_STORE_PATH):
        print("Building index from PDF...")
        index, chunks = build_index_from_pdf(DOCUMENT_PDF)
    else:
        index, docs = load_index_and_docs(FAISS_INDEX_PATH, DOC_STORE_PATH)
        chunks = [d["text"] for d in docs]

    # 2) load embedding model
    embedder = SentenceTransformer(EMBEDDING_MODEL_NAME, device=DEVICE)

    # If we just built index, save docs var
    if 'docs' not in locals():
        # create docs array
        import json
        docs = []
        with open(DOC_STORE_PATH, "r", encoding="utf-8") as f:
            for line in f:
                docs.append(json.loads(line.strip()))

    # 3) Load generator model (merged Llama)
    model, tokenizer = load_generator(MERGED_MODEL_DIR, TOKENIZER_DIR)
    # ensure model on device if needed
    # (FastLanguageModel.for_inference already optimizes / pins it)

    # 4) Example queries to test RAG
    sample_queries = [
        {"query": "Summarize the key principle established by Uma Devi (2006).", "gold": None},
        {"query": "Draft a short prayer seeking disability pension broadbanding for an ex-serviceman.", "gold": None},
        {"query": "What are the main changes introduced by the Industrial Relations Code, 2020?", "gold": None},
    ]

    # 5) Run RAG test
    results = run_rag_test(sample_queries, index, docs, embedder, model, tokenizer, top_k=5)

    # Optional: compute BERTScore vs golds if you have gold answers
    golds = [r["gold"] for r in results if r["gold"]]
    if golds:
        cands = [r["generated"] for r in results]
        print("BERTScore F1:", bertscore_f1(cands, golds))

    # Save results
    import json
    with open("rag_test_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print("Done. Results saved to rag_test_results.json")


/tmp/ipython-input-818535904.py:18: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [ ]:
from ragas.metrics import Faithfulness, ResponseRelevancy, LLMContextPrecisionWithoutReference
metrics = [Faithfulness(), ResponseRelevancy(), LLMContextPrecisionWithoutReference()]
